<a href="https://colab.research.google.com/github/SolSeyoum/Drought_Forecasting_TimeSeries/blob/master/Trend_extra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div>
<table style="width: 100%">
	<tr>
		<td>
		<table style="width: 100%">
			<tr>
                <td ><center><font size="5"><b>Module 49</b></font><center>
                <center><font size="6">Digital Innovations for Water Challenges</font><center></td>
			</tr>
			<tr>
                <td><center><font size="14">Notebook 4.3</font><center></td>
			</tr>
			<tr>
                <td><center><font size="6"><b>Trend from time series data</b></font><center></td>
			</tr>
		</table>
		</td>
		<td><center><img src='https://surfdrive.surf.nl/files/index.php/s/Xw3DUcZYO0lZIya/download?path=%2F&files=/ihe-delft-institute_unesco_fc-lr.jpg'></img></td>
	</tr>
</table>
</div>

<br>

For this exercise, we focus on time series data of near-surface air temperature over time. We will downlaod near-surface air temperature data over the Arctic, where increasing temperatures are particularly apparent.

We will download another subset of the dataset ERA5 monthly averaged data on single levels from 1979 to present.

This notebook is adapted from  <a href="https://ecmwf-projects.github.io/copernicus-training-c3s/reanalysis-climatology.html">Tutorial on Climatologies using Climate Data from C3S</a>

<style>
td, th {
   border: 1px solid white;
   border-collapse: collapse;
}
</style>
<table align="left">
  <tr>
    <th>Run the tutorial on Colab: </th>
    <th><a href="https://colab.research.google.com/github/ecmwf-projects/copernicus-training-c3s/blob/main/reanalysis-climatology.ipynb">
        <img src = "https://colab.research.google.com/assets/colab-badge.svg" alt = "Colab"></th>
  </tr>
</table>

<br>

## 1. Intall required packages

In [ ]:
!pip install cdsapi --quiet
!pip install cartopy --quiet
!pip install pymannkendall --quiet


#### Import libraries

We will be working with data in NetCDF format. To best handle this data we will use libraries for working with multidimensional arrays, in particular Xarray. We will also need libraries for plotting and viewing data, in this case we will use Matplotlib and Cartopy.

In [ ]:
# CDS API
import cdsapi

# Libraries for working with multidimensional arrays
import numpy as np
import xarray as xr
import pandas as pd

import pymannkendall as mk # Libarary to do trend analysis

# Libraries for plotting and visualising data
import matplotlib.path as mpath
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.feature as cfeature

# Disable warnings for data download via API
import urllib3
urllib3.disable_warnings()

#### Enter your CDS API key

In [ ]:

url = 'url: https://cds.climate.copernicus.eu/api'
key = input("your uid and key: ")
key = f"key: {key}"

with open('/root/.cdsapirc', 'w') as f:
    f.write('\n'.join([url, key]))

Specify a data directory in which we will download our data and all output files that we will generate:

In [ ]:
DATADIR = './'

#### Download data

In [ ]:
import cdsapi

dataset = "reanalysis-era5-single-levels-monthly-means"
request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "variable": ["2m_temperature"],
    "year": [
        "1979", "1980", "1981",
        "1982", "1983", "1984",
        "1985", "1986", "1987",
        "1988", "1989", "1990",
        "1991", "1992", "1993",
        "1994", "1995", "1996",
        "1997", "1998", "1999",
        "2000", "2001", "2002",
        "2003", "2004", "2005",
        "2006", "2007", "2008",
        "2009", "2010", "2011",
        "2012", "2013", "2014",
        "2015", "2016", "2017",
        "2018", "2019", "2020",
        "2021", "2022", "2023",
        "2024"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "time": ["00:00"],
    'area': [90, -180, 66.55, 180,],
    'data_format': 'netcdf_legacy',
}

client = cdsapi.Client()

# Retrieve the data and download it to the specified target path
client.retrieve(dataset, request).download(f'{DATADIR}era5_monthly_t2m_Arc.nc')

In [ ]:
t2m = f'{DATADIR}era5_monthly_t2m_Arc.nc'

In [ ]:
# Create Xarray Dataset
ds = xr.open_dataset(t2m)

Now we can query our newly created Xarray dataset ...

In [ ]:
ds

We see that the dataset has one variable called **"t2m"**, which stands for "2 metre temperature", and three coordinates of **longitude**, **latitude** and **time**.

In [ ]:
# Create Xarray Data Array
da = ds['t2m']

Let's view this data:

In [ ]:
da

#### Change temperature units from Kelvin to Celsius

Notice that the ERA-5 temperature data are in units of `Kelvin`, the base unit for temperature in the International System of Units (SI). If you want to convert the values from `Kelvin` to `degrees Celsius`, you have to subtract 273.15.

In [ ]:
da_degc = da - 273.15

If you inspect the characteristics of the data above, you see that when you convert the data values, the data array's Attributes are dropped. However, we want to keep the information provided by the Attributes and for this reason, we re-assign the attributes from the previous, unconverted object with the function assign_attrs(). Since the unit has changed, we assign a new unit measure to the units attribute.

In [ ]:
da_degc = da_degc.assign_attrs(da.attrs)
da_degc.attrs['units'] = '° C'

In [ ]:
da_degc


#### Plot data

Now, let us visualize one time step to get a better idea of the data. xarray offers built-in matplotlib functions that allow you to plot a `DataArray`. With the function `plot()`, you can easily plot e.g. the first time step of the loaded array.

In [ ]:
da_degc[0,:,:].plot()

An alternative to the built-in xarray plotting functions is to make use of a combination of the plotting libraries [matplotlib](https://matplotlib.org/) and [Cartopy](https://scitools.org.uk/cartopy/docs/latest/). One of Cartopy's key features is its ability to transform array data into different geographic projections. In combination with matplotlib, it is a very powerful way to create high-quality visualisations and animations. In later plots, we will make use of these libraries to produce more customised visualisations.

Let's view the first time step of this data. Notice that we change the projection from `ccrs.PlateCarree()` to `ccrs.Orthographic(central_latitude=90)` to better view the Arctic region. Note that we need to insert a `transform` keyword in the `pcolormesh` function to transform the data values into the orthographic projection:

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (8, 8), subplot_kw={'projection': ccrs.Orthographic(central_latitude=90)})

im = ax.pcolormesh(da_degc.longitude, da_degc.latitude, da_degc[0,:,:], transform = ccrs.PlateCarree(), cmap='coolwarm')
ax.gridlines(draw_labels=True, linewidth=1, color='gray', alpha=0.5, linestyle='--')
ax.set_title('Near-surface air temperature, Jan 1979', fontsize=16)
ax.coastlines(color='black')

cbar = fig.colorbar(im, fraction=0.04, pad=0.07)
cbar.set_label('° C')

fig.savefig(f'{DATADIR}ERA5_Arctic_t2m_Jan1979.png')

#### Aggregate over geographical lat/lon dimensions

We would like to analyse the time series of near-surface air temperature aggregated over the Arctic. To do this we need to average over the latitude and longitude dimensions.

A very important consideration however is that the gridded data cells do not all correspond to the same areas. The size covered by each data point varies as a function of latitude. We need to take this into account when averaging. One way to do this is to use the cosine of the latitude as a proxy for the varying sizes.

First, we calculate the weights by using the cosine of the latitude, then we apply these weights to the data array with the xarray function `weighted()`.

In [ ]:
weights = np.cos(np.deg2rad(da_degc.latitude))
weights.name = "weights"
da_degc_weighted = da_degc.weighted(weights)
da_degc_mean = da_degc_weighted.mean(["longitude", "latitude"])

Let us create a simple plot of this data to see how it looks:

In [ ]:
da_degc_mean.plot()

Notice that the `plot()` function now creates a graph of temperature as a function of time.

The trend in rising temperatures in the past decades is particularly noticable in the Arctic, and it is much easier to see this if we view the time series of yearly averages.

In [ ]:
da_degc_yearly = da_degc_mean.groupby('time.year').mean()

In [ ]:
da_degc_yearly.plot()

Here we can see a clear warming trend. However we need to do trend analysis to confirm if there is a trend or not. Here we can use Mann-Kendall test to do the trend analysis.

A typical way to view such a time series is to convert the absolute temperature values into anomalies with respect to a climate normal and view the time series as a bar chart. This can clearly highlight which years were on average warmer or cooler than the climate normal.

First let us do the trend analysis on the yearly timeseries.

In [ ]:
def print_MK_test_results(result):
  print(f"trend      = '{result.trend}'")
  print(f"h          = {result.h}")
  print(f"p          = {result.p:.3f}")
  print(f"z          = {result.z:.2f}")
  print(f"tau        = {result.Tau:.2f}")
  print(f"slope      = {result.slope:.2f}")
  print(f"intercept  = {result.intercept:.2f}")

In [ ]:
result_yearly = mk.original_test(da_degc_yearly)

# Print the results
print(result_yearly)
print_MK_test_results(result_yearly)



Now let us do monthly data

In [ ]:
result_monthly = mk.original_test(da_degc_mean)

# Print the results
print_MK_test_results(result_monthly)

Since the data has seasonality, the original MK test is not suitable. In this case we can try to use the MK test for seasonl data.


In [ ]:
# Seasonal Kendall Test (monthly seasonality)
result_seasonal = mk.seasonal_test(da_degc_mean, period=12)

print_MK_test_results(result_seasonal)

### How to Interpret the MK test results:

1. trend - Qualitative direction of change

| Value        | Meaning                  |
| ------------ | ------------------------ |
| `increasing` | Monotonic upward trend   |
| `decreasing` | Monotonic downward trend |
| `no trend`   | No significant trend     |


2. h (Hypothesis Test Result)

the null hypothesis (H0) - no monotonic trend is present in the data,
the alternate hypothesis (H1) - monotonic trend is present in the data

| Value        | Meaning                  |
| ------------ | ------------------------ |
| `True` | Reject null hypothesis. There is a statistically significant trend.   |
| `False` | annot reject null. Trend not statistically significant |


3. p (p-value) - Strength of evidence against “no trend”

| p-value  | Interpretation            |
| -------- | ------------------------- |
| p < 0.01 | Very strong evidence      |
| p < 0.05 | Statistically significant |
| p ≥ 0.05 | Not significant           |


4. z (Standard Normal Statistic) - Distance from “no trend” in standard deviations

| Z     | Meaning          |
| ----- | ---------------- |
| Z > 0 | Increasing trend |
| Z < 0 | Decreasing trend |


5. tau (Kendall’s Tau) - Strength of monotonic relationship
  Range of tau: −1 ≤ tau ≤ +1


6. slope (Sen’s Slope) - Magnitude of trend in units per time step


7. intercept - Value at time zero.



Exercise: Interpret the MK test results for the above three cases.

================================================================================
## Extra: How to clip and extract a timeseries for a point location

### Downlaod your data

Bounding box of Ethiopia as extracted from QGIS
 [32.9918000000000688,3.4066700000000765 : 47.9882400000000757,14.8836100000000897]

Remember when you define the area bound, the order is North, West, South and East

In [ ]:
import cdsapi

dataset = "reanalysis-era5-single-levels-monthly-means"
request = {
    "product_type": ["monthly_averaged_reanalysis"],
    "variable": ["2m_temperature"],
    "year": [
        "1979", "1980", "1981",
        "1982", "1983", "1984",
        "1985", "1986", "1987",
        "1988", "1989", "1990",
        "1991", "1992", "1993",
        "1994", "1995", "1996",
        "1997", "1998", "1999",
        "2000", "2001", "2002",
        "2003", "2004", "2005",
        "2006", "2007", "2008",
        "2009", "2010", "2011",
        "2012", "2013", "2014",
        "2015", "2016", "2017",
        "2018", "2019", "2020",
        "2021", "2022", "2023",
        "2024"
    ],
    "month": [
        "01", "02", "03",
        "04", "05", "06",
        "07", "08", "09",
        "10", "11", "12"
    ],
    "time": ["00:00"],
    'area': [14.9, 32.98, 3.4, 47.99,],
    'data_format': 'netcdf_legacy',
}

client = cdsapi.Client()

# Retrieve the data and download it to the specified target path
client.retrieve(dataset, request).download(f'{DATADIR}era5_monthly_t2m_Ethio.nc')

In [ ]:
t2m_et = f'{DATADIR}era5_monthly_t2m_Ethio.nc'
ds_eth = xr.open_dataset(t2m_et)
ds_eth

In [ ]:
ds_eth.t2m[0].plot()

## Clip the data to your region of interest.

In [ ]:
!pip install geopandas --quiet
!pip install rioxarray --quiet

In [ ]:
# upload shapefile of your area of interest


In [ ]:
import geopandas as gpd
import rioxarray as rio
region_shp = r'/content/Ethiopia_shp.geojson'
shp = gpd.read_file(region_shp).to_crs('EPSG:4326')
shp.plot()

In [ ]:
# clip the data using your area of interest
da_eth = ds_eth.t2m
da_eth = da_eth.rio.write_crs("EPSG:4326", inplace=False)
da_et_clipped = da_eth.rio.clip(shp.geometry.values, shp.crs, all_touched=True)
da_et_clipped[0].plot()

# Extract the data for your point location

In [ ]:
# lat lon of Adis Ababa
lat = 9.0192
lon = 38.7525

# Extract time series (nearest grid cell)
ts = da_et_clipped.sel(
    latitude=lat,
    longitude=lon,
    method="nearest"
)

print(ts)

In [ ]:
ts.plot()

In [ ]:
# Compute min and max over space for each time step
min_ts = da_et_clipped.min(dim=("latitude", "longitude"))
max_ts = da_et_clipped.max(dim=("latitude", "longitude"))

# Convert to pandas (optional)
min_ts_df = min_ts.to_pandas()
max_ts_df = max_ts.to_pandas()

min_ts_df.plot()

In [ ]:
max_ts_df.plot()

In [ ]:
plt.figure()

# Mean time series
plt.plot(ts["time"], ts, label="Mean")

# Min–max envelope
plt.fill_between(
    ts["time"],
    min_ts,
    max_ts,
    alpha=0.3,
    label="Min–Max range"
)

plt.xlabel("Time")
plt.ylabel("Variable units")
plt.title("Mean Time Series with Min–Max Bounds")
plt.legend()
plt.tight_layout()
plt.show()
